# Data exploration

In [ ]:
### Setup
!pip install -r requirements.txt

In [20]:
import pandas as pd
import awswrangler as wr
import matplotlib.pyplot as plt

### Getting data from s3

In [37]:
bucket_name = 'ds-benedict-tmp'
file_name = 'ds_interview_data.csv'
file_path = f's3://{bucket_name}/{file_name}'

df = wr.s3.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21960 entries, 0 to 21959
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   postcode                 21960 non-null  object 
 1   bedrooms                 21960 non-null  int64  
 2   main_type                21960 non-null  object 
 3   second_level_type        21938 non-null  object 
 4   footprint_area_sqm       21951 non-null  float64
 5   abshmin                  21943 non-null  float64
 6   absh2                    21941 non-null  float64
 7   abshmax                  21943 non-null  float64
 8   relh2                    21941 non-null  float64
 9   relhmax                  21943 non-null  float64
 10  internal_floor_area_sqm  21806 non-null  float64
 11  chain_1_hash             17152 non-null  object 
dtypes: float64(7), int64(1), object(4)
memory usage: 2.0+ MB
